# Google Directions API

* Save request response (json) as pickles to cache.
* Each request (one trip / route) can take up to 25 waypoints before broken up into 2 separate requests.

In [1]:
import dotenv
import geopandas as gpd
import googlemaps
import glob
import os
import pandas as pd
import pickle

import setup_gmaps
import utils

from calitp.storage import get_fs
fs = get_fs()

dotenv.load_dotenv("_env")

GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"]

DATA_PATH = "./gmaps_cache/"
GCS_FILE_PATH = f"{utils.GCS_FILE_PATH}gmaps_cache/"

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
'''
df = setup_gmaps.make_gmaps_df()

# Create identifier, as we loop through each row to make gmaps request
# if there are errors, can find those later, without having to rerun requests
# use this identifier in pickle naming
df["identifier"] = df.calitp_itp_id.astype(str).str.cat(
                    [df.route_id, df.trip_id, df.shape_id], sep ="__")
                    
print(type(df))
df.to_parquet("./gmaps_df.parquet")
'''

df = pd.read_parquet("./gmaps_df.parquet")
df.head()

,calitp_itp_id,route_id,trip_id,shape_id,day_name,trip_departure,destination,origin,departure_in_one_year,waypoints,identifier
0,4,10,13277020,shp-10-10,Thursday,2022-01-06 07:08:00,"[37.670239, -122.08718]","[37.721266, -122.160201]",2023-01-05 07:08:00,"[[37.72521, -122.156367], [37.720677, -122.150...",4__10__13277020__shp-10-10
1,4,12,2883020,shp-12-14,Thursday,2022-01-06 22:58:00,"[37.879358, -122.301843]","[37.79442, -122.271546]",2023-01-05 22:58:00,"[[37.796581, -122.275884], [37.805037, -122.27...",4__12__2883020__shp-12-14
2,4,14,3396020,shp-14-14,Thursday,2022-01-06 06:14:00,"[37.775518, -122.225458]","[37.787142, -122.195417]",2023-01-05 06:14:00,"[[37.783773, -122.201934], [37.778549, -122.20...",4__14__3396020__shp-14-14
3,4,18,10001020,shp-18-18,Thursday,2022-01-06 06:54:00,"[37.884226, -122.299531]","[37.79716, -122.265807]",2023-01-05 06:54:00,"[[37.80017, -122.273479], [37.806506, -122.269...",4__18__10001020__shp-18-18
4,4,19,947020,shp-19-16,Thursday,2022-01-06 22:20:00,"[37.77534, -122.225114]","[37.803658, -122.276263]",2023-01-05 22:20:00,"[[37.801197, -122.273313], [37.788309, -122.27...",4__19__947020__shp-19-16


In [3]:
gmaps = googlemaps.Client(key=GOOGLE_API_KEY)

In [4]:
# Check if there are any identifiers already cached
# Drop those from our df
FILES = [f for f in glob.glob(f"{DATA_PATH}*.p")]
filenames = []
for f in FILES:
    file = f.strip(f'{DATA_PATH}').strip('.p')
    filenames.append(file)

print(f"# rows: {len(df)}")
df = df[~df.identifier.isin(filenames)]
print(f"# rows after local caches included: {len(df)}")

# rows: 1619
# rows after local caches included: 1617


In [5]:
origin = df.origin.tolist()
destination = df.destination.tolist()
departures = df.departure_in_one_year.tolist()
waypoints = df.waypoints.tolist()
identifiers = df.identifier.tolist()

In [6]:
# https://stackoverflow.com/questions/25052980/use-pickle-to-save-dictionary-in-python
def save_request_pickle(my_list, name):
    # result comes back as a list, but add [0] and it's a dict
    my_dict = my_list[0]
    
    # Save as local pickle
    pickle.dump(my_dict, open(f"{DATA_PATH}{name}.p", "wb"))
    
    # pickle can't be written directly to GCS. Put the pickle object there. 
    fs.put(f"./{DATA_PATH}{name}.p", f"{GCS_FILE_PATH}{name}.p")
    print(f"Saved {name}")
    
def open_request_pickle(name):
    # Download object from GCS bucket (pickle can't be read directly)
    gcs_pickle = fs.get(f"{GCS_FILE_PATH}{name}.p", f"{DATA_PATH}{name}.p")
    my_dict = pickle.load(open(f"{DATA_PATH}{name}.p", "rb"))
    
    return my_dict

In [8]:
for i, (o, d) in enumerate(zip(origin, destination)):
    result = gmaps.directions(
        o, d, 
        mode='driving', 
        departure_time=departures[i], 
        waypoints=[f"via:{lat},{lon}" for lat, lon in waypoints[i]]
    )
    
    save_request_pickle(result, identifiers[i])

Saved 4__14__3396020__shp-14-14
Saved 4__18__10001020__shp-18-18
Saved 4__19__947020__shp-19-16
Saved 4__1T__6142020__shp-1T-51
Saved 4__20__6901020__shp-20-16
Saved 4__200__1035020__shp-200-60
Saved 4__21__7085020__shp-21-20
Saved 4__210__7205020__shp-210-07
Saved 4__215__5096020__shp-215-15
Saved 4__216__8382020__shp-216-16
Saved 4__217__8749020__shp-217-10
Saved 4__232__11766020__shp-232-10
Saved 4__239__1480020__shp-239-08
Saved 4__251__1875020__shp-251-59
Saved 4__28__5737020__shp-28-58
Saved 4__29__227020__shp-29-30
Saved 4__33__13157020__shp-33-69
Saved 4__34__5593020__shp-34-56
Saved 4__35__8205020__shp-35-61
Saved 4__36__5319020__shp-36-09
Saved 4__376__5401020__shp-376-08
Saved 4__39__9303020__shp-39-53
Saved 4__40__3480020__shp-40-61
Saved 4__41__4226020__shp-41-04
Saved 4__45__3934020__shp-45-53
Saved 4__46L__625020__shp-46L-02
Saved 4__51A__5786020__shp-51A-17
Saved 4__51B__2361020__shp-51B-14
Saved 4__52__2149020__shp-52-13
Saved 4__54__311020__shp-54-03
Saved 4__56__3550

FileNotFoundError: [Errno 2] No such file or directory: './gmaps_cache/106__Shellmound/Powell__12_Trip10_SP_COVID__p_899325.p'

In [ ]:
'''
# Use via waypoints to get duration_in_traffic
# longest travel time compared to sum(waypoints)
via = gmaps.directions(origin, destination, 
                 mode='driving', 
                 departure_time=departures, 
                 waypoints=[f"via:{lat},{lon}" for lat, lon in waypoints]
                )
'''